In [ ]:
import hydra
from lib.utils.config import load_config
from lib.data.datamodule import DPHMDataModule
from lib.data.scheduler import CoarseToFineScheduler

cfg = load_config(
    "optimize",
    overrides=[
        "model.init_mode=kinect",
        "model.vertices_mask=full",
        "model.optimize_frames=10",
        "data=kinect",
        "data.scale=8"
        "data.batch_size=10",
        "data.start_frame_idx=19",
    ],
)
datamodule: DPHMDataModule = hydra.utils.instantiate(cfg.data, devie="cuda")
datamodule.setup()
c2fs: CoarseToFineScheduler = hydra.utils.instantiate(cfg.scheduler.coarse2fine)

In [ ]:
import timeit

def benchmark_update_dataset():
    c2fs.schedule_dataset(datamodule=datamodule, iter_step=0)

number = 1
time = timeit.timeit(benchmark_update_dataset, number=number) / number
print(f"Execution time: {time} seconds")

In [ ]:
datamodule.dataset.load_color()

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

dataset = datamodule.dataset
def process_frame(frame_idx):
    return dataset._load_color(frame_idx)

normal = []

with ThreadPoolExecutor() as executor:
    futures = {
        executor.submit(process_frame, frame_idx): frame_idx
        for frame_idx in dataset.iter_frame_idx()
    }

    for future in as_completed(futures):
        frame_idx = futures[future]
        try:
            _normal = future.result()
            normal.append((frame_idx, _normal))
        except Exception as e:
            print(f"Error processing frame {frame_idx}: {e}")

# Sort the results to ensure the frames are in the original order
# normal.sort(key=lambda x: x[0])
# normal = [normal for _, normal in normal]